In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
def pregunta_01():
    """
    Carga de datos.
    -------------------------------------------------------------------------------------
    """

    # Lea el archivo `amazon_cells_labelled.tsv` y cree un DataFrame usando pandas.
    # Etiquete la primera columna como `msg` y la segunda como `lbl`. Esta función
    # retorna el dataframe con las dos columnas.
    tabla = pd.read_csv("amazon_cells_labelled.tsv",
        sep='\t',
        header=None,
        names=['msg','lbl'],
    )
    df=pd.DataFrame(tabla)

    # Separe los grupos de mensajes etiquetados y no etiquetados.
    df_tagged = df[(df["lbl"]>=0)]
    df_untagged = df[df["lbl"].isnull()]

    x_tagged = df_tagged[df_tagged["msg"] ==0.0];x_tagged
    y_tagged = df_tagged[df_tagged["lbl"] ==1.0];y_tagged

    x_untagged = df_untagged["msg"]
    y_untagged = df_untagged["lbl"]

    # Retorne los grupos de mensajes
    return (x_tagged, y_tagged, x_untagged, y_untagged)






(Empty DataFrame
 Columns: [msg, lbl]
 Index: [],
                                                     msg  lbl
 2                           Good case, Excellent value.  1.0
 5                                Great for the jawbone.  1.0
 11                                    The mic is great.  1.0
 17          If you are Razr owner...you must have this!  1.0
 24                      And the sound quality is great.  1.0
 ...                                                 ...  ...
 2742                                 Excellent product.  1.0
 2748  It is the best charger I have seen on the mark...  1.0
 2749                                  SWEETEST PHONE!!!  1.0
 2750             :-)Oh, the charger seems to work fine.  1.0
 2751  It fits so securely that the ear hook does not...  1.0
 
 [500 rows x 2 columns],
 0        I try not to adjust the volume setting to avoi...
 3              I thought Motorola made reliable products!.
 4                               Battery for Motorola Razr.

In [27]:
def pregunta_02():
    """
    Preparación de los conjuntos de datos.
    -------------------------------------------------------------------------------------
    """

    # Importe train_test_split
    import numpy as np
    from sklearn.model_selection import train_test_split

    # Cargue los datos generados en la pregunta 01.
    x_tagged, y_tagged, x_untagged, y_untagged = pregunta_01()

    # Divida los datos de entrenamiento y prueba. La semilla del generador de números
    # aleatorios es 12345. Use el 10% de patrones para la muestra de prueba.
    x_train, x_test, y_train, y_test = train_test_split(x_tagged, y_tagged, test_size=0.1,random_state=12345)

    # Retorne `X_train`, `X_test`, `y_train` y `y_test`
    return x_train, x_test, y_train, y_test





ValueError: Found input variables with inconsistent numbers of samples: [0, 500]

In [18]:
def pregunta_03():
    """
    Construcción de un analizador de palabras
    -------------------------------------------------------------------------------------
    """
    # Importe el stemmer de Porter
    # Importe CountVectorizer
    from sklearn.feature_extraction.text import CountVectorizer
    from nltk.stem.porter  import PorterStemmer

    # Cree un stemeer que use el algoritmo de Porter.
    stemmer = PorterStemmer()
    vectorr= CountVectorizer(analyzer="word",token_pattern=r"(?u)\b[a-zA-z][a-zA-z]+\b", lowercase=True)
    # Cree una instancia del analizador de palabras (build_analyzer)
    analyzer = vectorr.build_analyzer()

    # Retorne el analizador de palabras
    return lambda x: (stemmer.stem(w) for w in analyzer(x))

In [22]:
def pregunta_04():
    """
    Especificación del pipeline y entrenamiento
    -------------------------------------------------------------------------------------
    """

    # Importe CountVetorizer
    # Importe GridSearchCV
    # Importe Pipeline
    # Importe BernoulliNB
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.model_selection import GridSearchCV
    from sklearn.pipeline import Pipeline
    from sklearn.naive_bayes import BernoulliNB

    # Cargue las variables.
    x_train, _, y_train, _ = pregunta_02()

    # Obtenga el analizador de la pregunta 3.
    analyzer = pregunta_03()

    # Cree una instancia de CountVectorizer que use el analizador de palabras
    # de la pregunta 3. Esta instancia debe retornar una matriz binaria. El
    # límite superior para la frecuencia de palabras es del 100% y un límite
    # inferior de 5 palabras. Solo deben analizarse palabras conformadas por
    # letras.
    countVectorizer = countVectorizer(analyzer=analyzer(),lowercase=True,stop_words="english",token_pattern=r"(?u)\b[a-zA-Z][a-zA-Z]+\b",binary=False,max_df=1.0,min_df=5)

    # Cree un pipeline que contenga el CountVectorizer y el modelo de BernoulliNB.
    pipeline = pipeline(steps=[("countvectorizer", countVectorizer),
            ("BernoulliNB", BernoulliNB()),
        ],
    )

    # Defina un diccionario de parámetros para el GridSearchCV. Se deben
    # considerar 10 valores entre 0.1 y 1.0 para el parámetro alpha de
    # BernoulliNB.
    param_grid = {"BernoulliNB_alpha": np.arange(0.1, 1.1, 10)}

    # Defina una instancia de GridSearchCV con el pipeline y el diccionario de
    # parámetros. Use cv = 5, y "accuracy" como métrica de evaluación
    gridSearchCV = GridSearchCV(estimator=pipeline,param_grid=param_grid,
        cv=5,
        scoring="accuracy",
        refit=True,
        return_train_score=True,
    )

    # Búsque la mejor combinación de regresores
    gridSearchCV.fit(x_train, y_train)

    # Retorne el mejor modelo
    return gridSearchCV

In [26]:
def pregunta_05():
    """
    Evaluación del modelo
    -------------------------------------------------------------------------------------
    """

    # Importe confusion_matrix
    from sklearn.metrics import confusion_matrix

    # Obtenga el pipeline de la pregunta 3.
    gridSearchCV = pregunta_04()

    # Cargue las variables.
    X_train, X_test, y_train, y_test = pregunta_02()

    # Evalúe el pipeline con los datos de entrenamiento usando la matriz de confusion.
    cm_train = confusion_matrix(
        y_true=y_train,
        y_pred=gridSearchCV.predict(X_train),
    )

    cm_test = confusion_matrix(
        y_true=y_test,
        y_pred=gridSearchCV.predict(X_test),
    )

    # Retorne la matriz de confusion de entrenamiento y prueba
    return cm_train, cm_test

In [ ]:
def pregunta_06():
    """
    Pronóstico
    -------------------------------------------------------------------------------------
    """

    # Obtenga el pipeline de la pregunta 3.
    gridSearchCV = pregunta_04()

    # Cargue los datos generados en la pregunta 01.
    x_tagged,y_tagged, x_untagged,y_untagged = pregunta_01()

    # pronostique la polaridad del sentimiento para los datos
    # no etiquetados
    y_untagged_pred = gridSearchCV.predict(x_untagged)

    # Retorne el vector de predicciones
    return y_untagged_pred